# Homework : Neural Machine Translation
***
## [Objective]
Pytorch De-En Translator



In [ ]:
!pip uninstall spacy -y
!pip install -U spacy

Found existing installation: spacy 3.1.1
Uninstalling spacy-3.1.1:
  Successfully uninstalled spacy-3.1.1
  Using cached spacy-3.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.4 MB)


In [11]:
!pip install torchtext==0.8.1

     |████████████████████████████████| 7.0 MB 13.9 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.7.1 which is incompatible.


In [32]:
import jieba
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import numpy as np
import pandas as pd
import spacy
import random
from torchtext.data.metrics import bleu_score
from pprint import pprint
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

In [2]:
!mkdir ./data
!mkdir ./data/multi30k
!python -m spacy download en
!ls ./data/multi30k -al
spacy_english = spacy.load("en_core_web_sm")
!ls ./data/multi30k -al

mkdir: cannot create directory ‘./data’: File exists
mkdir: cannot create directory ‘./data/multi30k’: File exists
2021-07-21 14:21:16.441771: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.6 MB 165 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
total 8
drwxr-xr-x 2 root root 4096 Jul 21 13:25 .
drwxr-xr-x 3 root root 4096 Jul 21 13:25 ..
total 8
drwxr-xr-x 2 root root 4096 Jul 21 13:25 .
drwxr-xr-x 3 root root 4096 Jul 21 13:25 ..


In [3]:
!python -m spacy download de
spacy_de = spacy.load("de_core_news_sm")
!ls ./data/multi30k -al

2021-07-21 14:21:24.289498: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
⚠ As of spaCy v3.0, shortcuts like 'de' are deprecated. Please use the
full pipeline package name 'de_core_news_sm' instead.
     |████████████████████████████████| 18.8 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
total 8
drwxr-xr-x 2 root root 4096 Jul 21 13:25 .
drwxr-xr-x 3 root root 4096 Jul 21 13:25 ..


In [11]:

def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)]

def tokenize_english(text):
  return [token.text for token in spacy_english.tokenizer(text)]

### Sample Run ###

sample_text = "I love machine learning"
print(tokenize_english(sample_text))

german = Field(tokenize=tokenize_de, lower=True,
               init_token="<sos>", eos_token="<eos>")

english = Field(tokenize=tokenize_english, lower=True,
               init_token="<sos>", eos_token="<eos>")

train_data, valid_data, test_data = Multi30k.splits(exts = (".de", ".en"),
                                                    fields=(german, english))

german.build_vocab(train_data, max_size=10000, min_freq=3)
english.build_vocab(train_data, max_size=10000, min_freq=3)

print(f"Unique tokens in source (german) vocabulary: {len(german.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(english.vocab)}")


['I', 'love', 'machine', 'learning']


/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


Unique tokens in source (german) vocabulary: 5374
Unique tokens in target (en) vocabulary: 4556


In [19]:
word_2_idx = dict(english.vocab)
idx_2_word = {}
for k,v in word_2_idx.items():
  idx_2_word[v] = k

TypeError: ignored

In [12]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

print(train_data[5].__dict__.keys())
pprint(train_data[5].__dict__.values())

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
dict_keys(['src', 'trg'])
dict_values([['ein', 'mann', 'in', 'grün', 'hält', 'eine', 'gitarre', ',', 'während', 'der', 'andere', 'mann', 'sein', 'hemd', 'ansieht', '.'], ['a', 'man', 'in', 'green', 'holds', 'a', 'guitar', 'while', 'the', 'other', 'man', 'observes', 'his', 'shirt', '.']])


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                                                      batch_size = BATCH_SIZE, 
                                                                      sort_within_batch=True,
                                                                      sort_key=lambda x: len(x.src),
                                                                      device = device)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [13]:
count = 0
max_len_eng = []
max_len_ger = []
for data in train_data:
  max_len_ger.append(len(data.src))
  max_len_eng.append(len(data.trg))
  if count < 10 :
    print("German - ",*data.src, " Length - ", len(data.src))
    print("English - ",*data.trg, " Length - ", len(data.trg))
    print()
  count += 1

print("Maximum Length of English sentence {} and German sentence {} in the dataset".format(max(max_len_eng),max(max_len_ger)))
print("Minimum Length of English sentence {} and German sentence {} in the dataset".format(min(max_len_eng),min(max_len_ger)))

German -  zwei junge weiße männer sind im freien in der nähe vieler büsche .  Length -  13
English -  two young , white males are outside near many bushes .  Length -  11

German -  mehrere männer mit schutzhelmen bedienen ein antriebsradsystem .  Length -  8
English -  several men in hard hats are operating a giant pulley system .  Length -  12

German -  ein kleines mädchen klettert in ein spielhaus aus holz .  Length -  10
English -  a little girl climbing into a wooden playhouse .  Length -  9

German -  ein mann in einem blauen hemd steht auf einer leiter und putzt ein fenster .  Length -  15
English -  a man in a blue shirt is standing on a ladder cleaning a window .  Length -  15

German -  zwei männer stehen am herd und bereiten essen zu .  Length -  10
English -  two men are at the stove preparing food .  Length -  9

German -  ein mann in grün hält eine gitarre , während der andere mann sein hemd ansieht .  Length -  16
English -  a man in green holds a guitar while the other

In [15]:
count = 0
for data in train_iterator:
  if count < 1 :
    print("Shapes", data.src.shape, data.trg.shape)
    print()
    print("German - ",*data.src, " Length - ", len(data.src))
    print()
    print("English - ",*data.trg, " Length - ", len(data.trg))
    temp_ger = data.src
    temp_eng = data.trg
    count += 1

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Shapes torch.Size([18, 32]) torch.Size([20, 32])

German -  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([ 16,   4,  16,   4,   4,   4,  45, 249,   4,   4,  21,   4,  16,   4,
          4,   4,   4,   4,  16,   4,   4,   4,   4,   4,   4,   4,  28,   4,
          4,   4,  45,   4], device='cuda:0') tensor([ 30,  23, 632,  26,  15, 682,  61, 123, 119, 301, 229,   9,  49,   9,
         15,  23,  23, 158,  19,  26,  55,  23,  15,  23,   9,  15, 264,   9,
          9,   9,   9,  29], device='cuda:0') tensor([  17,   34,   17,   11,  148,   51,  107,  433,  237,   61,    9,    6,
           6,  205,   10,   32,    9, 2454,   17,   35,   10,   15,  186,  528,
          88,    6,   68,    6,   13,   98,    6,  175], device='cuda:0') tensor([  77, 1506,  611,   24,    4,   34,    0,   16,    6, 1382,  371, 1046,
          85,    8,   31,   22,   10,   10,   31,  501,   36,   13,   10,   32,
          79

In [16]:
temp_eng_idx = (temp_eng).cpu().detach().numpy()
temp_ger_idx = (temp_ger).cpu().detach().numpy()

In [17]:

df_eng_idx = pd.DataFrame(data = temp_eng_idx, columns = [str("S_")+str(x) for x in np.arange(1, 33)])
df_eng_idx.index.name = 'Time Steps'
df_eng_idx.index = df_eng_idx.index + 1 
# df_eng_idx.to_csv('/content/idx.csv')
df_eng_idx


,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2,16,4,16,4,4,4,46,251,4,4,21,4,16,4,4,4,4,4,16,4,4,4,4,4,4,4,28,4,4,4,46,4
3,30,24,639,26,14,683,61,127,123,299,233,9,50,9,14,24,24,161,19,26,55,24,14,24,9,14,269,9,9,9,9,29
4,17,34,17,11,151,53,112,439,237,61,9,6,6,212,10,33,9,2452,17,35,10,14,189,535,89,6,69,6,13,100,6,180
5,78,1518,617,25,4,34,0,16,6,1387,375,1049,86,8,32,22,10,10,32,504,36,13,10,33,80,4,7,4,4,6,1454,1128
6,40,27,236,35,615,140,20,12,62,22,6,148,270,7,71,263,2190,45,83,21,49,4,714,11,4,590,220,178,0,43,11,40
7,7,139,167,6,8,4,7,155,11,4,4,11,51,192,18,219,82,4,4,86,4,132,44,941,0,10,7,314,158,12,46,4
8,39,15,72,4,44,1768,72,414,26,2173,253,4,18,85,4,579,14,183,159,68,162,23,418,3872,206,36,9,42,3433,4,6,2265
9,8,58,8,384,172,28,35,4,338,89,243,898,849,151,535,3460,13,2452,11,15,13,10,28,2426,15,6,6,109,933,291,728,474


In [18]:
df_eng_word = pd.DataFrame(columns = [str("S_")+str(x) for x in np.arange(1, 33)])
df_eng_word = df_eng_idx.replace(idx_2_word)
# df_eng_word.to_csv('/content/Words.csv')
df_eng_word

NameError: ignored

## EncoderLSTM



In [21]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)
    
    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

  # Shape of x (26, 32) [Sequence_length, batch_size]
  def forward(self, x):

    # Shape -----------> (26, 32, 300) [Sequence_length , batch_size , embedding dims]
    embedding = self.dropout(self.embedding(x))
    
    # Shape --> outputs (26, 32, 1024) [Sequence_length , batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size]
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)

    return hidden_state, cell_state

input_size_encoder = len(german.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(5374, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


## DecoderLSTM


In [22]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)

    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

    # Shape -----------> (1024, 4556) [embedding dims, hidden size, num layers]
    self.fc = nn.Linear(hidden_size, output_size)

  # Shape of x (32) [batch_size]
  def forward(self, x, hidden_state, cell_state):

    # Shape of x (1, 32) [1, batch_size]
    x = x.unsqueeze(0)

    # Shape -----------> (1, 32, 300) [1, batch_size, embedding dims]
    embedding = self.dropout(self.embedding(x))

    # Shape --> outputs (1, 32, 1024) [1, batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))

    # Shape --> predictions (1, 32, 4556) [ 1, batch_size , output_size]
    predictions = self.fc(outputs)

    # Shape --> predictions (32, 4556) [batch_size , output_size]
    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

input_size_decoder = len(english.vocab)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.5
output_size = len(english.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(4556, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=4556, bias=True)
)


In [23]:
for batch in train_iterator:
  print(batch.src.shape)
  print(batch.trg.shape)
  break

x = batch.trg[1]
print(x)

torch.Size([37, 32])
torch.Size([37, 32])
tensor([  16,   48,    4,    4,    4,   48,    4,    4,    4,  176,    4,    4,
           4,   16,    4,    4,    4,    4,  176,    4,    4,   24,    4,    4,
          21,    4, 1720,    4,    4,    4,    4,    4], device='cuda:0')


/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


# Sequence to Sequence

In [24]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):
    # Shape - Source : (10, 32) [(Sentence length german + some padding), Number of Sentences]
    batch_size = source.shape[1]

    # Shape - Source : (14, 32) [(Sentence length English + some padding), Number of Sentences]
    target_len = target.shape[0]
    target_vocab_size = len(english.vocab)
    
    # Shape --> outputs (14, 32, 5766) 
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    # Shape --> (hs, cs) (2, 32, 1024) ,(2, 32, 1024) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
    hidden_state, cell_state = self.Encoder_LSTM(source)

    # Shape of x (32 elements)
    x = target[0] # Trigger token <SOS>

    for i in range(1, target_len):
      # Shape --> output (32, 5766) 
      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output
      best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
      x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

    # Shape --> outputs (14, 32, 5766) 
    return outputs


In [25]:
# Hyperparameters

learning_rate = 0.001
writer = SummaryWriter(f"runs/loss_plot")
step = 0

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [26]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(5374, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(4556, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=4556, bias=True)
  )
)

In [33]:
def translate_sentence(model, sentence, german, english, device, max_length=50):
    spacy_ger = spacy.load("de_core_news_sm")

    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)
    text_to_indices = [german.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [english.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

def bleu(data, model, german, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

def checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss):
    print('saving')
    print()
    state = {'model': model,'best_loss': best_loss,'epoch': epoch,'rng_state': torch.get_rng_state(), 'optimizer': optimizer.state_dict(),}
    torch.save(state, './checkpoint-NMT')
    torch.save(model.state_dict(),'./checkpoint-NMT-SD')

In [30]:
epoch_loss = 0.0
num_epochs = 100
best_loss = 999999
best_epoch = -1
sentence1 = "ein mann in einem blauen hemd steht auf einer leiter und putzt ein fenster"
ts1  = []

for epoch in range(num_epochs):
  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  model.eval()
  translated_sentence1 = translate_sentence(model, sentence1, german, english, device, max_length=50)
  print(f"Translated example sentence 1: \n {translated_sentence1}")
  ts1.append(translated_sentence1)

  model.train(True)
  for batch_idx, batch in enumerate(train_iterator):
    input = batch.src.to(device)
    target = batch.trg.to(device)

    # Pass the input and target for model's forward method
    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    # Clear the accumulating gradients
    optimizer.zero_grad()

    # Calculate the loss value for every epoch
    loss = criterion(output, target)

    # Calculate the gradients for weights & biases using back-propagation
    loss.backward()

    # Clip the gradient value is it exceeds > 1
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    # Update the weights values using the gradients we calculated using bp 
    optimizer.step()
    step += 1
    epoch_loss += loss.item()
    writer.add_scalar("Training loss", loss, global_step=step)

  if epoch_loss < best_loss:
    best_loss = epoch_loss
    best_epoch = epoch
    checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss) 
    if ((epoch - best_epoch) >= 10):
      print("no improvement in 10 epochs, break")
      break
  print("Epoch_Loss - {}".format(loss.item()))
  print()
  
print(epoch_loss / len(train_iterator))

# score = bleu(test_data[1:100], model, german, english, device)
# print(f"Bleu score {score*100:.2f}")

Epoch - 1 / 100
Translated example sentence 1: 
 ['sit', 'moving', 'moving', 'bungee', 'drivers', 'force', 'moped', 'gloved', 'hammock', 'gloved', 'faced', 'faced', 'itself', 'hammock', 'daylight', 'daylight', 'meditating', 'meditating', 'meditating', 'gloved', 'curly', 'curly', 'gloved', 'gloved', 'hammock', 'gloved', 'hotdog', 'gloved', 'hotdog', 'rolled', 'rolled', 'rolled', 'hairnet', 'moped', 'designer', 'designer', 'designer', 'unloading', 'elbow', 'darkened', 'ally', 'artist', 'rolled', 'rolled', 'moped', 'moped', 'remote', 'remote', 'feet', 'feet']


/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


saving

Epoch_Loss - 2.6146411895751953

Epoch - 2 / 100
Translated example sentence 1: 
 ['there', 'in', 'the', 'air', 'with', '<unk>', ',', ',', 'and', 'the', '<unk>', 'and', '<unk>', '.', '<eos>']
Epoch_Loss - 3.4478724002838135

Epoch - 3 / 100
Translated example sentence 1: 
 ['here', 'with', 'the', 'ball', 'in', 'the', 'and', 'is', 'is', 'is', 'to', 'to', 'it', '.', '<eos>']
Epoch_Loss - 1.9869029521942139

Epoch - 4 / 100
Translated example sentence 1: 
 ['cheerleaders', 'with', 'the', 'field', 'in', 'the', ',', ',', 'is', 'is', 'is', 'is', 'up', '.', '<eos>']
Epoch_Loss - 1.5970442295074463

Epoch - 5 / 100
Translated example sentence 1: 
 ['here', 'with', 'the', 'street', 'in', 'his', ',', 'and', 'is', 'is', 'is', 'his', 'them', '.', '<eos>']
Epoch_Loss - 1.3171850442886353

Epoch - 6 / 100
Translated example sentence 1: 
 ['worker', 'with', 'the', 'number', 'on', 'top', ',', 'black', 'and', 'is', 'is', 'his', 'them', '.', '<eos>']
Epoch_Loss - 1.2669719457626343

Epoch - 7 / 

In [18]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
pegasus_model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-reddit_tifu').to(device)
pegasus_tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-reddit_tifu')

input_text = ' '.join(input_text.split())
input_text = 'we went to a vineyard for the holiday weekend . there were pigs there . it was my first time seeing such pigs . our friend wore a funny hat that caught everyone's attention . he was the subject for many jokes . later , we all gathered at the long table to eat and drink wine . '
batch = pegasus_tokenizer(input_text, truncation=True, padding='longest', return_tensors="pt").to(device)

summary_ids = pegasus_model.generate(**batch,
                                    num_beams=6,
                                    num_return_sequences=1,
                                    no_repeat_ngram_size = 2,
                                    length_penalty = 1,
                                    min_length = 30,
                                    max_length = 128,
                                    early_stopping = True)

output = [pegasus_tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)]
output

[['went to a vineyard and saw pigs . friend wore a funny hat that caught everyones attention and was the subject of a lot of jokes.']]